In [1]:
import numpy as np
import re

In [2]:
def loadPCAWeightsFromSV(fName,nbits=12):
    pca_weights=np.zeros(30*1024).reshape(30,32,32)

    regex_in=re.compile(f"^assign weights_in\[(\d*)\]\[(\d*)\]\[(\d*)\] = {nbits}'b(\d*);")

    with open(fName,'r') as _file:
        for line in _file:
            m=regex_in.match(line)
            if m:
                i,j,k,v=m.groups()
                v=int(v,2)
                v = (-1*(v>>11)*2**11 + (v&0x7ff))/2**11
                pca_weights[int(i)][int(j)][int(k)]=v

    return pca_weights

In [3]:
def loadPCAInputsFromSV(fName):
    pcaInputs=np.zeros(1024).reshape(32,32)

    regex_in=re.compile("^assign pca_data_in_10b\[(\d*)\]\[(\d*)\] = 10'b(\d*);")

    with open(fName,'r') as _file:
        for line in _file:
            m=regex_in.match(line)
            if m:
                i,j,v=m.groups()
                v=int(v,2)
                pcaInputs[int(i)][int(j)]=v

    return pcaInputs

In [4]:
def toTwosCompliment(x,N=11,M=0,doRounding=False):
    if doRounding:
        y=round(x*2**N)
    else:
        y=int(x*2**N)
    if y < 0:
        return f"{M+N+1}'b1"+f"{(2**(N+M)+y):0b}".zfill(M+N)
    else:
        return f"{M+N+1}'b{y:0b}"

toTwosCompliment=np.vectorize(toTwosCompliment)

In [5]:
R_inv_float=np.load('mandrill_140/inv_R_matrix_float32.npy')[:,:30]*32


In [6]:
pcaWeights=loadPCAWeightsFromSV('svData/PCA_weightsInput_Original.sv')
pcaWeights=pcaWeights.reshape(30,1024).T


In [7]:
pcaInputs=loadPCAInputsFromSV('svData/testVectors_0.sv')


In [8]:
np.matmul(pcaInputs.flatten(),pcaWeights)

array([-0.45751953, -0.26806641,  0.05322266,  1.60595703,  0.78955078,
        1.06445312, -2.06738281,  1.08837891,  0.51220703, -0.20507812,
       -0.80273438,  0.52929688, -0.59130859,  0.39746094,  0.01855469,
        0.49316406,  0.09619141,  1.17871094,  1.14648438, -0.07714844,
        1.0078125 ,  0.22607422, -0.79980469,  1.68408203,  0.76025391,
       -0.80175781,  0.97851562,  0.35839844,  0.59716797,  0.17919922])

In [9]:
np.matmul(pcaInputs.flatten(),pcaWeights)*2**11

array([ -937.,  -549.,   109.,  3289.,  1617.,  2180., -4234.,  2229.,
        1049.,  -420., -1644.,  1084., -1211.,   814.,    38.,  1010.,
         197.,  2414.,  2348.,  -158.,  2064.,   463., -1638.,  3449.,
        1557., -1642.,  2004.,   734.,  1223.,   367.])

In [10]:
output=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int)>>10)
output

array([-1, -1,  0,  3,  1,  2, -5,  2,  1, -1, -2,  1, -2,  0,  0,  0,  0,
        2,  2, -1,  2,  0, -2,  3,  1, -2,  1,  0,  1,  0])

In [11]:
twosOutput=toTwosCompliment(output,0,6)
twosOutput

array(["7'b1111111", "7'b1111111", "7'b0", "7'b11", "7'b1", "7'b10",
       "7'b1111011", "7'b10", "7'b1", "7'b1111111", "7'b1111110", "7'b1",
       "7'b1111110", "7'b0", "7'b0", "7'b0", "7'b0", "7'b10", "7'b10",
       "7'b1111111", "7'b10", "7'b0", "7'b1111110", "7'b11", "7'b1",
       "7'b1111110", "7'b1", "7'b0", "7'b1", "7'b0"], dtype='<U10')

In [12]:
for n in range(0,601,100):
    lines = ""
    with open(f'svData/testVectors_{n}.sv','r') as _file:
        #drop old 19 b pca values
        for l in _file:
            if '// 19b PCA outputs' in l:
                continue
            if 'output from original PCA weights' in l:
                continue
            if 'pca_output' in l:
                continue
            lines += l

    print(n)
    pcaInputs=loadPCAInputsFromSV(f'svData/testVectors_{n}.sv')
    
    output7=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int)>>10)
    twosOutput=toTwosCompliment(output7,0,6)
    
    lines+="// 7-bit output from original PCA weights (from Panpan)\n\n"
    for i,v in enumerate(twosOutput):
        lines += f"assign pca_output_orig_7b[{i}] = {v};\n"
    lines += '\n\n'
    
    output19=((np.matmul(pcaInputs.flatten(),pcaWeights)*2**11).astype(int))
    twosOutput=toTwosCompliment(output19,0,18)
    
    lines+="// 19-bit output from original PCA weights (from Panpan)\n\n"
    for i,v in enumerate(twosOutput):
        lines += f"assign pca_output_orig_19b[{i}] = {v};\n"
    lines += '\n\n'
    
    with open(f'svData/testVectors_{n}.sv','w') as _file:
        _file.write(lines)

    l=', '.join([f'{x/2:.1f}' for x in output7])
    with open(f'svData/dataOut_{n}_Q7.6.dat','w') as _file:
        _file.write(l)

    l=', '.join([f'{x/2**11:.11f}' for x in output19])
    with open(f'svData/dataOut_{n}_Q19.8.dat','w') as _file:
        _file.write(l)

0
100
200
300
400
500
600
